In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib import rc,rcParams

rc('text', usetex=True)
rc('axes', linewidth=2)
rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']

ipykernel_launcher:9: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.


In [2]:
n_vals = 8
model_prop_val_map = {
    '0040000': 'r',
    '0050000': 'g',
    '0060000': 'b'
}
model_prop = 'c'

def rand_between(size, a, b):
    return np.random.rand(*size) * (b - a) + a

scores = np.concatenate(
    (rand_between((n_vals, 1), 67, 78), rand_between((n_vals, 1), 13, 22)),
    axis=1
)
model_suffixes = tuple(model_prop_val_map.keys())
models = [np.random.choice(model_suffixes) for _ in range(n_vals)]
models

['0040000',
 '0050000',
 '0050000',
 '0050000',
 '0040000',
 '0050000',
 '0040000',
 '0040000']

In [3]:
scores

array([[77.50324122, 21.78932612],
       [75.00347405, 17.08076062],
       [77.05685769, 17.59673066],
       [73.46754013, 20.57689615],
       [76.36008136, 15.2061852 ],
       [73.9386418 , 15.01386264],
       [70.46885635, 16.99318596],
       [72.89069947, 21.6201931 ]])

In [4]:
def build_prop_dict(vals, vals_prop_map, prop_name):
    prop_vals = [vals_prop_map[v] for v in vals]
    prop_dict = {prop_name: prop_vals}
    return prop_dict

def build_axis_label(text, units=None):
    label = rf'${text}$'
    if units is not None:
        label += f' [{units}]'
    return label

x_label = 'MOTA'
y_label = 'MOTP'
x_units = y_units = None

x_label_formatted = build_axis_label(x_label, x_units)
y_label_formatted = build_axis_label(y_label, y_units)
print(x_label_formatted)
size = 20 ** 2

prop_dict = build_prop_dict(models, model_prop_val_map, model_prop)

fig, ax = plt.subplots(figsize=(8, 6), nrows=1, ncols=1)
ax.set_title(r'$\textbf{Tracker Comparison}$')
ax.set_xlabel(x_label_formatted)
ax.set_ylabel(y_label_formatted)

ax.scatter(scores[:, 0], scores[:, 1], s=size, marker=r'$\bigtriangleup$', **prop_dict)
fig.show()

$MOTA$


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 576x432 with 1 Axes>